In [7]:
import math, pickle
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
./experiments/scripts/rfcn_end2end_ohem.sh 0 ResNet-101 pascal_voc &

In [2]:
%run ./tools/test_net.py --gpu 0 --def models/pascal_voc/ResNet-101/rfcn_end2end/test_agnostic.prototxt --net /home/ubuntu/py-R-FCN/output/rfcn_end2end_ohem/voc_2007_trainval/resnet101_rfcn_ohem_iter_30000.caffemodel --imdb voc_2007_test --cfg experiments/cfgs/rfcn_end2end_ohem.yml

Called with args:
Namespace(caffemodel='/home/ubuntu/py-R-FCN/output/rfcn_end2end_ohem/voc_2007_trainval/resnet101_rfcn_ohem_iter_30000.caffemodel', cfg_file='experiments/cfgs/rfcn_end2end_ohem.yml', comp_mode=False, gpu_id=0, imdb_name='voc_2007_test', max_per_image=400, prototxt='models/pascal_voc/ResNet-101/rfcn_end2end/test_agnostic.prototxt', rpn_file=None, set_cfgs=None, vis=False, wait=True)
Using config:
{'DATA_DIR': '/home/ubuntu/py-R-FCN/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'rfcn_end2end_ohem',
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MODELS_DIR': '/home/ubuntu/py-R-FCN/models/pascal_voc',
 'PIXEL_MEANS': array([[[ 102.9801,  115.9465,  122.7717]]]),
 'RNG_SEED': 3,
 'ROOT_DIR': '/home/ubuntu/py-R-FCN',
 'TEST': {'AGNOSTIC': True,
          'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'selective_search',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_

In [9]:
with open('output/rfcn_end2end_ohem/voc_2007_test/resnet101_rfcn_ohem_iter_30000/detections_full_AGNOSTICnms_resnet101_rfcn_ohem_iter_30000.pkl','rb') as f:
    detections_full_AGNOSTICnms = pickle.load(f) 